In [1]:
import pandas as pd
import os

### Preprocessing Price Tables

In [2]:
df_price_de = pd.read_csv('raw_data/day_ahead_price_de.csv')
df_price_fr = pd.read_csv('raw_data/day_ahead_price_fr.csv')
df_price_at = pd.read_csv('raw_data/day_ahead_price_at.csv')

In [5]:
df_price_de['hour_index'] = df_price_de['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_price_de['date'] = df_price_de['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_price_de['date'] = pd.to_datetime(df_price_de['date'])
df_price_de['datetime_utc'] = pd.to_datetime(df_price_de['datetime'], utc=True)
df_price_de['day_of_week'] = df_price_de['date'].dt.dayofweek
df_price_de['day'] = df_price_de['date'].dt.day
df_price_de['month'] = df_price_de['date'].dt.month
df_price_de['year'] = df_price_de['date'].dt.year
df_price_de.rename(columns={'price': 'price_de'}, inplace=True)

In [6]:
df_price_fr['hour_index'] = df_price_fr['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_price_fr['date'] = df_price_fr['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_price_fr['date'] = pd.to_datetime(df_price_fr['date'])
df_price_fr['datetime_utc'] = pd.to_datetime(df_price_fr['datetime'], utc=True)
df_price_fr['day_of_week'] = df_price_fr['date'].dt.dayofweek
df_price_fr['day'] = df_price_fr['date'].dt.day
df_price_fr['month'] = df_price_fr['date'].dt.month
df_price_fr['year'] = df_price_fr['date'].dt.year
df_price_fr.rename(columns={'price': 'price_fr'}, inplace=True)

In [7]:
df_price_at['hour_index'] = df_price_at['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_price_at['date'] = df_price_at['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_price_at['date'] = pd.to_datetime(df_price_at['date'])
df_price_at['datetime_utc'] = pd.to_datetime(df_price_at['datetime'], utc=True)
df_price_at['day_of_week'] = df_price_at['date'].dt.dayofweek
df_price_at['day'] = df_price_at['date'].dt.day
df_price_at['month'] = df_price_at['date'].dt.month
df_price_at['year'] = df_price_at['date'].dt.year
df_price_at.rename(columns={'price': 'price_at'}, inplace=True)

In [8]:
df_price_de = df_price_de[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'price_de']]
df_price_fr = df_price_fr[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'price_fr']]
df_price_at = df_price_at[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'price_at']]

In [9]:
def adding_multiple_lags(df, column, lags):
    df_copy = df.copy()
    for lag in lags:
        df_copy[f'{column}_lag_{lag}'] = df_copy[column].shift(lag)
    return df_copy

In [10]:
df_price_de = adding_multiple_lags(df_price_de, 'price_de', [24, 48, 168, 336])
df_price_fr = adding_multiple_lags(df_price_fr, 'price_fr', [24])
df_price_at = adding_multiple_lags(df_price_at, 'price_at', [24])

In [13]:
def adding_multiple_rolling_windows(df, column, rolling_windows):
    df_copy = df.copy()
    for rolling_window in rolling_windows:
        df_copy[f'{column}_avg_{rolling_window}'] = df_copy[column].rolling(rolling_window).mean()
    return df_copy

In [14]:
df_price_de = adding_multiple_rolling_windows(df_price_de, 'price_de', [24, 168])
df_price_fr = adding_multiple_rolling_windows(df_price_fr, 'price_fr', [24, 168])
df_price_at = adding_multiple_rolling_windows(df_price_at, 'price_at', [24, 168])

In [17]:
df_price_merged = pd.merge(df_price_de, pd.merge(df_price_at, df_price_fr, on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner'), on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner').sort_values(by=['datetime_utc'])

### Preprocessing DE Load Table

In [18]:
df_load_de = pd.read_csv('raw_data/day_ahead_load_de.csv')
df_load_fr = pd.read_csv('raw_data/day_ahead_load_fr.csv')
df_load_at = pd.read_csv('raw_data/day_ahead_load_at.csv')

In [19]:
df_load_de['hour_index'] = df_load_de['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_load_de['date'] = df_load_de['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_load_de['date'] = pd.to_datetime(df_load_de['date'])
df_load_de['datetime_utc'] = pd.to_datetime(df_load_de['datetime'], utc=True)
df_load_de['day_of_week'] = df_load_de['date'].dt.dayofweek
df_load_de['day'] = df_load_de['date'].dt.day
df_load_de['month'] = df_load_de['date'].dt.month
df_load_de['year'] = df_load_de['date'].dt.year
df_load_de.rename(columns={'totalloadvalue': 'load_de'}, inplace=True)

In [20]:
df_load_de = df_load_de[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'load_de']]

In [21]:
df_load_fr['hour_index'] = df_load_fr['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_load_fr['date'] = df_load_fr['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_load_fr['date'] = pd.to_datetime(df_load_fr['date'])
df_load_fr['datetime_utc'] = pd.to_datetime(df_load_fr['datetime'], utc=True)
df_load_fr['day_of_week'] = df_load_fr['date'].dt.dayofweek
df_load_fr['day'] = df_load_fr['date'].dt.day
df_load_fr['month'] = df_load_fr['date'].dt.month
df_load_fr['year'] = df_load_fr['date'].dt.year
df_load_fr.rename(columns={'totalloadvalue': 'load_fr'}, inplace=True)

In [22]:
df_load_fr = df_load_fr[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'load_fr']]

In [23]:
df_load_at['hour_index'] = df_load_at['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_load_at['date'] = df_load_at['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_load_at['date'] = pd.to_datetime(df_load_at['date'])
df_load_at['datetime_utc'] = pd.to_datetime(df_load_at['datetime'], utc=True)
df_load_at['day_of_week'] = df_load_at['date'].dt.dayofweek
df_load_at['day'] = df_load_at['date'].dt.day
df_load_at['month'] = df_load_at['date'].dt.month
df_load_at['year'] = df_load_at['date'].dt.year
df_load_at.rename(columns={'totalloadvalue': 'load_at'}, inplace=True)

In [24]:
df_load_at = df_load_at[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'load_at']]

In [25]:
df_load_de = adding_multiple_lags(df_load_de, 'load_de', [24, 48, 168, 336])
df_load_fr = adding_multiple_lags(df_load_fr, 'load_fr', [24])
df_load_at = adding_multiple_lags(df_load_at, 'load_at', [24])

In [28]:
df_load_merged = pd.merge(df_load_de, pd.merge(df_load_at, df_load_fr, on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner'), on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner').sort_values(by=['datetime_utc'])

### Processing Generation Table

In [29]:
df_gen_de = pd.read_csv('raw_data/day_ahead_gen_de.csv')
df_gen_fr = pd.read_csv('raw_data/day_ahead_gen_fr.csv')
df_gen_at = pd.read_csv('raw_data/day_ahead_gen_at.csv')

In [30]:
df_gen_de['hour_index'] = df_gen_de['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_gen_de['date'] = df_gen_de['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_gen_de['date'] = pd.to_datetime(df_gen_de['date'])
df_gen_de['datetime_utc'] = pd.to_datetime(df_gen_de['datetime'], utc=True)
df_gen_de['day_of_week'] = df_gen_de['date'].dt.dayofweek
df_gen_de['day'] = df_gen_de['date'].dt.day
df_gen_de['month'] = df_gen_de['date'].dt.month
df_gen_de['year'] = df_gen_de['date'].dt.year
df_gen_de.rename(columns={'scheduledgeneration': 'gen_de'}, inplace=True)

In [31]:
df_gen_de = df_gen_de[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'gen_de']]

In [32]:
df_gen_fr['hour_index'] = df_gen_fr['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_gen_fr['date'] = df_gen_fr['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_gen_fr['date'] = pd.to_datetime(df_gen_fr['date'])
df_gen_fr['datetime_utc'] = pd.to_datetime(df_gen_fr['datetime'], utc=True)
df_gen_fr['day_of_week'] = df_gen_fr['date'].dt.dayofweek
df_gen_fr['day'] = df_gen_fr['date'].dt.day
df_gen_fr['month'] = df_gen_fr['date'].dt.month
df_gen_fr['year'] = df_gen_fr['date'].dt.year
df_gen_fr.rename(columns={'scheduledgeneration': 'gen_fr'}, inplace=True)

In [33]:
df_gen_fr = df_gen_fr[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'gen_fr']]

In [34]:
df_gen_at['hour_index'] = df_gen_at['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_gen_at['date'] = df_gen_at['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_gen_at['date'] = pd.to_datetime(df_gen_at['date'])
df_gen_at['datetime_utc'] = pd.to_datetime(df_gen_at['datetime'], utc=True)
df_gen_at['day_of_week'] = df_gen_at['date'].dt.dayofweek
df_gen_at['day'] = df_gen_at['date'].dt.day
df_gen_at['month'] = df_gen_at['date'].dt.month
df_gen_at['year'] = df_gen_at['date'].dt.year
df_gen_at.rename(columns={'scheduledgeneration': 'gen_at'}, inplace=True)

In [35]:
df_gen_at = df_gen_at[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'gen_at']]

In [36]:
df_gen_de = adding_multiple_lags(df_gen_de, 'gen_de', [24, 48, 168, 336])
df_gen_fr = adding_multiple_lags(df_gen_fr, 'gen_fr', [24])
df_gen_at = adding_multiple_lags(df_gen_at, 'gen_at', [24])

In [39]:
df_gen_merged = pd.merge(df_gen_de, pd.merge(df_gen_at, df_gen_fr, on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner'), on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner').sort_values(by=['datetime_utc'])

### Processing Wind Off- and On-shore and Solar Tables

In [40]:
df_wind_off_de = pd.read_csv('raw_data/day_ahead_wind_off_de.csv')

In [41]:
df_wind_off_de['hour_index'] = df_wind_off_de['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_wind_off_de['date'] = df_wind_off_de['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_wind_off_de['date'] = pd.to_datetime(df_wind_off_de['date'])
df_wind_off_de['datetime_utc'] = pd.to_datetime(df_wind_off_de['datetime'], utc=True)
df_wind_off_de['day_of_week'] = df_wind_off_de['date'].dt.dayofweek
df_wind_off_de['day'] = df_wind_off_de['date'].dt.day
df_wind_off_de['month'] = df_wind_off_de['date'].dt.month
df_wind_off_de['year'] = df_wind_off_de['date'].dt.year
df_wind_off_de.rename(columns={'aggregatedgenerationforecast': 'windoff_de'}, inplace=True)

In [42]:
df_wind_off_de = df_wind_off_de[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'windoff_de']]

In [45]:
df_wind_off_de = adding_multiple_lags(df_wind_off_de, 'windoff_de', [24])

In [46]:
df_wind_on_de = pd.read_csv('raw_data/day_ahead_wind_on_de.csv')

In [47]:
df_wind_on_de['hour_index'] = df_wind_on_de['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_wind_on_de['date'] = df_wind_on_de['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_wind_on_de['date'] = pd.to_datetime(df_wind_on_de['date'])
df_wind_on_de['datetime_utc'] = pd.to_datetime(df_wind_on_de['datetime'], utc=True)
df_wind_on_de['day_of_week'] = df_wind_on_de['date'].dt.dayofweek
df_wind_on_de['day'] = df_wind_on_de['date'].dt.day
df_wind_on_de['month'] = df_wind_on_de['date'].dt.month
df_wind_on_de['year'] = df_wind_on_de['date'].dt.year
df_wind_on_de.rename(columns={'aggregatedgenerationforecast': 'windon_de'}, inplace=True)

In [48]:
df_wind_on_de = df_wind_on_de[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'windon_de']]

In [51]:
df_wind_on_de = adding_multiple_lags(df_wind_on_de, 'windon_de', [24])

In [52]:
df_solar_de = pd.read_csv('raw_data/day_ahead_solar_de.csv')

In [53]:
df_solar_de['hour_index'] = df_solar_de['datetime'].str.extract(r' (\d{2}):\d{2}:\d{2}').astype(int)
df_solar_de['date'] = df_solar_de['datetime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df_solar_de['date'] = pd.to_datetime(df_solar_de['date'])
df_solar_de['datetime_utc'] = pd.to_datetime(df_solar_de['datetime'], utc=True)
df_solar_de['day_of_week'] = df_solar_de['date'].dt.dayofweek
df_solar_de['day'] = df_solar_de['date'].dt.day
df_solar_de['month'] = df_solar_de['date'].dt.month
df_solar_de['year'] = df_solar_de['date'].dt.year
df_solar_de.rename(columns={'aggregatedgenerationforecast': 'solar_de'}, inplace=True)

In [54]:
df_solar_de = df_solar_de[['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week', 'solar_de']]

In [55]:
df_solar_de = adding_multiple_lags(df_solar_de, 'solar_de', [24])

In [56]:
df_renew_merged = pd.merge(df_wind_off_de, pd.merge(df_wind_on_de, df_solar_de, on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner'), on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner').sort_values(by=['datetime_utc'])

#### Due to the rolling windows and lags, some NaN values appear at the start of time series.

In [57]:
df_price_merged

,datetime,datetime_utc,date,day,month,year,hour_index,day_of_week,price_de,price_de_lag_24,...,price_de_avg_24,price_de_avg_168,price_at,price_at_lag_24,price_at_avg_24,price_at_avg_168,price_fr,price_fr_lag_24,price_fr_avg_24,price_fr_avg_168
0,2018-10-01 00:00:00+02,2018-09-30 22:00:00+00:00,2018-10-01,1,10,2018,0,0,59.53,NaN,...,NaN,NaN,59.53,NaN,NaN,NaN,59.53,62.86,55.607500,62.431012
1,2018-10-01 01:00:00+02,2018-09-30 23:00:00+00:00,2018-10-01,1,10,2018,1,0,56.10,NaN,...,NaN,NaN,56.10,NaN,NaN,NaN,56.10,57.20,55.561667,62.478155
2,2018-10-01 02:00:00+02,2018-10-01 00:00:00+00:00,2018-10-01,1,10,2018,2,0,51.41,NaN,...,NaN,NaN,51.41,NaN,NaN,NaN,51.41,54.87,55.417500,62.505357
3,2018-10-01 03:00:00+02,2018-10-01 01:00:00+00:00,2018-10-01,1,10,2018,3,0,47.38,NaN,...,NaN,NaN,47.38,NaN,NaN,NaN,47.38,49.66,55.322500,62.559821
4,2018-10-01 04:00:00+02,2018-10-01 02:00:00+00:00,2018-10-01,1,10,2018,4,0,47.59,NaN,...,NaN,NaN,47.59,NaN,NaN,NaN,47.59,42.54,55.532917,62.611488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45212,2023-11-27 19:00:00+01,2023-11-27 18:00:00+00:00,2023-11-27,27,11,2023,19,0,117.12,143.78,...,118.861250,98.050179,129.20,140.09,120.909583,102.035119,127.33,146.10,119.420833,102.264286
45213,2023-11-27 20:00:00+01,2023-11-27 19:00:00+00:00,2023-11-27,27,11,2023,20,0,100.61,135.91,...,117.390417,97.934524,117.77,135.91,120.153750,102.025655,118.44,135.91,118.692917,102.254762
45214,2023-11-27 21:00:00+01,2023-11-27 20:00:00+00:00,2023-11-27,27,11,2023,21,0,94.46,122.41,...,116.225833,97.859821,102.90,122.41,119.340833,102.001190,102.80,122.41,117.875833,102.229702
45215,2023-11-27 22:00:00+01,2023-11-27 21:00:00+00:00,2023-11-27,27,11,2023,22,0,95.20,118.99,...,115.234583,97.783333,97.10,118.99,118.428750,101.936012,100.00,118.99,117.084583,102.181786


In [58]:
df_load_merged

,datetime,datetime_utc,date,day,month,year,hour_index,day_of_week,load_de,load_de_lag_24,load_de_lag_48,load_de_lag_168,load_de_lag_336,load_at,load_at_lag_24,load_fr,load_fr_lag_24
0,2022-01-01 01:00:00+01,2022-01-01 00:00:00+00:00,2022-01-01,1,1,2022,1,5,41862.6675,NaN,NaN,NaN,NaN,5697.0,NaN,49200.0,NaN
1,2022-01-01 02:00:00+01,2022-01-01 01:00:00+00:00,2022-01-01,1,1,2022,2,5,40026.6875,NaN,NaN,NaN,NaN,5590.0,NaN,47950.0,NaN
2,2022-01-01 03:00:00+01,2022-01-01 02:00:00+00:00,2022-01-01,1,1,2022,3,5,38789.9925,NaN,NaN,NaN,NaN,5413.0,NaN,44950.0,NaN
3,2022-01-01 04:00:00+01,2022-01-01 03:00:00+00:00,2022-01-01,1,1,2022,4,5,38035.3500,NaN,NaN,NaN,NaN,5403.0,NaN,43150.0,NaN
4,2022-01-01 05:00:00+01,2022-01-01 04:00:00+00:00,2022-01-01,1,1,2022,5,5,38080.7600,NaN,NaN,NaN,NaN,5467.0,NaN,42850.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16671,2023-11-28 19:00:00+01,2023-11-28 18:00:00+00:00,2023-11-28,28,11,2023,19,1,68287.8275,67039.8200,55247.1125,65782.2400,64749.5675,8629.0,8430.0,70700.0,70450.0
16672,2023-11-28 20:00:00+01,2023-11-28 19:00:00+00:00,2023-11-28,28,11,2023,20,1,65667.7600,64622.6850,53718.7450,63637.6150,62683.3375,8133.0,7898.0,67250.0,66650.0
16673,2023-11-28 21:00:00+01,2023-11-28 20:00:00+00:00,2023-11-28,28,11,2023,21,1,61869.1575,60766.2900,51508.2700,60012.7700,58930.7550,7627.0,7338.0,63650.0,62650.0
16674,2023-11-28 22:00:00+01,2023-11-28 21:00:00+00:00,2023-11-28,28,11,2023,22,1,58327.2075,57360.6475,50111.4500,56835.6225,55508.2225,7415.0,7073.0,61900.0,60550.0


In [59]:
df_gen_merged

,datetime,datetime_utc,date,day,month,year,hour_index,day_of_week,gen_de,gen_de_lag_24,gen_de_lag_48,gen_de_lag_168,gen_de_lag_336,gen_at,gen_at_lag_24,gen_fr,gen_fr_lag_24
0,2022-01-01 01:00:00+01,2022-01-01 00:00:00+00:00,2022-01-01,1,1,2022,1,5,50246.30,NaN,NaN,NaN,NaN,6775.7,NaN,55286.0,NaN
1,2022-01-01 02:00:00+01,2022-01-01 01:00:00+00:00,2022-01-01,1,1,2022,2,5,49655.51,NaN,NaN,NaN,NaN,6826.1,NaN,52546.5,NaN
2,2022-01-01 03:00:00+01,2022-01-01 02:00:00+00:00,2022-01-01,1,1,2022,3,5,47618.88,NaN,NaN,NaN,NaN,6835.8,NaN,49277.0,NaN
3,2022-01-01 04:00:00+01,2022-01-01 03:00:00+00:00,2022-01-01,1,1,2022,4,5,45778.35,NaN,NaN,NaN,NaN,6841.9,NaN,48473.0,NaN
4,2022-01-01 05:00:00+01,2022-01-01 04:00:00+00:00,2022-01-01,1,1,2022,5,5,44719.31,NaN,NaN,NaN,NaN,6773.4,NaN,48259.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16745,2023-11-29 19:00:00+01,2023-11-29 18:00:00+00:00,2023-11-29,29,11,2023,19,2,62773.99,57841.85,67725.92,67205.89,64284.45,12037.3,12881.7,69724.5,66852.5
16746,2023-11-29 20:00:00+01,2023-11-29 19:00:00+00:00,2023-11-29,29,11,2023,20,2,59304.16,57975.94,64837.21,64378.39,62871.49,11684.9,11507.4,67162.5,65453.0
16747,2023-11-29 21:00:00+01,2023-11-29 20:00:00+00:00,2023-11-29,29,11,2023,21,2,55026.34,56861.96,61277.96,59774.70,61061.06,11347.4,10070.8,64758.5,62724.5
16748,2023-11-29 22:00:00+01,2023-11-29 21:00:00+00:00,2023-11-29,29,11,2023,22,2,53021.65,54238.72,59481.88,55996.64,58853.77,8983.0,9464.4,64066.5,61883.5


In [60]:
df_renew_merged

,datetime,datetime_utc,date,day,month,year,hour_index,day_of_week,windoff_de,windoff_de_lag_24,windon_de,windon_de_lag_24,solar_de,solar_de_lag_24
0,2022-09-01 00:00:00+02,2022-08-31 22:00:00+00:00,2022-09-01,1,9,2022,0,3,1730.5400,NaN,8869.1275,NaN,0.0,NaN
1,2022-09-01 01:00:00+02,2022-08-31 23:00:00+00:00,2022-09-01,1,9,2022,1,3,1421.3100,NaN,8154.0000,NaN,0.0,NaN
2,2022-09-01 02:00:00+02,2022-09-01 00:00:00+00:00,2022-09-01,1,9,2022,2,3,1284.2475,NaN,7500.1075,NaN,0.0,NaN
3,2022-09-01 03:00:00+02,2022-09-01 01:00:00+00:00,2022-09-01,1,9,2022,3,3,1169.0325,NaN,6859.0950,NaN,0.0,NaN
4,2022-09-01 04:00:00+02,2022-09-01 02:00:00+00:00,2022-09-01,1,9,2022,4,3,1016.7150,NaN,6311.6575,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10915,2023-11-29 19:00:00+01,2023-11-29 18:00:00+00:00,2023-11-29,29,11,2023,19,2,956.9600,3624.7300,11211.8400,5648.4275,0.0,0.0
10916,2023-11-29 20:00:00+01,2023-11-29 19:00:00+00:00,2023-11-29,29,11,2023,20,2,826.1350,3477.6850,10125.9200,6621.6025,0.0,0.0
10917,2023-11-29 21:00:00+01,2023-11-29 20:00:00+00:00,2023-11-29,29,11,2023,21,2,689.0900,3143.9400,8953.2350,8064.3875,0.0,0.0
10918,2023-11-29 22:00:00+01,2023-11-29 21:00:00+00:00,2023-11-29,29,11,2023,22,2,602.8725,2954.1775,7845.9175,9454.0200,0.0,0.0


### Merging all tables together

In [61]:
df_merged = pd.merge(df_price_merged, pd.merge(df_load_merged, pd.merge(df_gen_merged, df_renew_merged, on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner'), on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner'), on=['datetime', 'datetime_utc', 'date', 'day', 'month', 'year', 'hour_index', 'day_of_week'], how='inner').sort_values(by=['datetime_utc'])

In [62]:
df_merged

,datetime,datetime_utc,date,day,month,year,hour_index,day_of_week,price_de,price_de_lag_24,...,gen_at,gen_at_lag_24,gen_fr,gen_fr_lag_24,windoff_de,windoff_de_lag_24,windon_de,windon_de_lag_24,solar_de,solar_de_lag_24
0,2022-09-01 00:00:00+02,2022-08-31 22:00:00+00:00,2022-09-01,1,9,2022,0,3,490.77,531.48,...,4404.8,3595.2,32687.5,32510.0,1730.5400,NaN,8869.1275,NaN,0.0,NaN
1,2022-09-01 01:00:00+02,2022-08-31 23:00:00+00:00,2022-09-01,1,9,2022,1,3,484.36,523.31,...,4266.3,3503.6,32147.5,32016.5,1421.3100,NaN,8154.0000,NaN,0.0,NaN
2,2022-09-01 02:00:00+02,2022-09-01 00:00:00+00:00,2022-09-01,1,9,2022,2,3,486.34,541.87,...,4291.7,3473.3,30197.0,30318.5,1284.2475,NaN,7500.1075,NaN,0.0,NaN
3,2022-09-01 03:00:00+02,2022-09-01 01:00:00+00:00,2022-09-01,1,9,2022,3,3,480.02,498.16,...,4333.0,3449.2,30378.5,29886.0,1169.0325,NaN,6859.0950,NaN,0.0,NaN
4,2022-09-01 04:00:00+02,2022-09-01 02:00:00+00:00,2022-09-01,1,9,2022,4,3,495.18,541.83,...,4395.5,3443.6,30594.5,29982.5,1016.7150,NaN,6311.6575,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10865,2023-11-27 19:00:00+01,2023-11-27 18:00:00+00:00,2023-11-27,27,11,2023,19,0,117.12,143.78,...,8048.2,9631.7,67186.0,66263.0,5655.6600,912.4075,19610.9375,8586.9775,0.0,0.0
10866,2023-11-27 20:00:00+01,2023-11-27 19:00:00+00:00,2023-11-27,27,11,2023,20,0,100.61,135.91,...,7182.3,7971.9,65092.0,65548.5,5723.7850,769.3625,19850.2150,8735.0100,0.0,0.0
10867,2023-11-27 21:00:00+01,2023-11-27 20:00:00+00:00,2023-11-27,27,11,2023,21,0,94.46,122.41,...,6889.9,6812.3,61485.0,63025.0,5770.7025,654.9925,19866.5850,8688.0825,0.0,0.0
10868,2023-11-27 22:00:00+01,2023-11-27 21:00:00+00:00,2023-11-27,27,11,2023,22,0,95.20,118.99,...,6623.8,6340.0,59942.5,61792.5,5851.5325,575.8400,19787.7525,8613.6325,0.0,0.0


#### Saving the file, cutting off the beginning of the time series containing NaN values.

In [63]:
df_merged.query("date >= '2022-11-01'").to_csv('raw_data/df_electricity_merged.csv', index=False)

In [64]:
pd.read_csv('raw_data/df_electricity_merged.csv')

,datetime,datetime_utc,date,day,month,year,hour_index,day_of_week,price_de,price_de_lag_24,...,gen_at,gen_at_lag_24,gen_fr,gen_fr_lag_24,windoff_de,windoff_de_lag_24,windon_de,windon_de_lag_24,solar_de,solar_de_lag_24
0,2022-11-01 00:00:00+01,2022-10-31 23:00:00+00:00,2022-11-01,1,11,2022,0,1,80.65,129.38,...,4079.2,2991.0,40284.0,36238.0,5331.8525,2384.8325,16063.7425,6744.5025,0.0,0.0
1,2022-11-01 01:00:00+01,2022-11-01 00:00:00+00:00,2022-11-01,1,11,2022,1,1,76.68,119.09,...,4027.1,2942.3,39246.0,36353.5,5206.9600,2536.8800,16839.0400,6414.3500,0.0,0.0
2,2022-11-01 02:00:00+01,2022-11-01 01:00:00+00:00,2022-11-01,1,11,2022,2,1,55.26,116.32,...,3933.9,2940.3,39366.5,36030.0,5295.7975,2461.7675,17616.8775,5744.3775,0.0,0.0
3,2022-11-01 03:00:00+01,2022-11-01 02:00:00+00:00,2022-11-01,1,11,2022,3,1,37.00,114.60,...,3766.6,2869.9,37003.5,35798.5,5410.9475,2317.1050,17984.3850,5127.0875,0.0,0.0
4,2022-11-01 04:00:00+01,2022-11-01 03:00:00+00:00,2022-11-01,1,11,2022,4,1,38.76,115.07,...,3585.8,2988.1,34949.5,36479.0,5552.2900,2176.6850,18902.0375,4400.1125,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9401,2023-11-27 19:00:00+01,2023-11-27 18:00:00+00:00,2023-11-27,27,11,2023,19,0,117.12,143.78,...,8048.2,9631.7,67186.0,66263.0,5655.6600,912.4075,19610.9375,8586.9775,0.0,0.0
9402,2023-11-27 20:00:00+01,2023-11-27 19:00:00+00:00,2023-11-27,27,11,2023,20,0,100.61,135.91,...,7182.3,7971.9,65092.0,65548.5,5723.7850,769.3625,19850.2150,8735.0100,0.0,0.0
9403,2023-11-27 21:00:00+01,2023-11-27 20:00:00+00:00,2023-11-27,27,11,2023,21,0,94.46,122.41,...,6889.9,6812.3,61485.0,63025.0,5770.7025,654.9925,19866.5850,8688.0825,0.0,0.0
9404,2023-11-27 22:00:00+01,2023-11-27 21:00:00+00:00,2023-11-27,27,11,2023,22,0,95.20,118.99,...,6623.8,6340.0,59942.5,61792.5,5851.5325,575.8400,19787.7525,8613.6325,0.0,0.0
